In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup 
%matplotlib inline

In [2]:
df_full=pd.read_csv('data/Seasons_Stats.csv')

In [3]:
df_full.drop(columns='Unnamed: 0', inplace=True)

In [4]:
# Year - Season
# Player - name
# Pos - Position
# Age - Age
# Tm - Team
# G - Games
# GS - Games Started
# MP - Minutes Played
# PER - Player Efficiency Rating
# TS% - True Shooting %
# 3PAr - 3-Point Attempt Rate
# FTr - Free Throw Rate
# ORB% - Offensive Rebound Percentage
# DRB% - Defensive Rebound Percentage
# TRB% - Total Rebound Percentage
# AST% - Assist Percentage
# STL% - Steal Percentage
# BLK% - Block Percentage
# TOV% - Turnover Percentage
# USG% - Usage Percentage
# blanlempty
# OWS - Offensive Win Shares
# DWS - Defensive Win Shares
# WS - Win Shares
# WS/48Win Shares Per 48 Minutes
# blank2
# OBPM - Offensive Box Plus/Minus
# DBPM - Defensive Box Plus/Minus
# BPM - Box Plus/Minus
# VORPValue Over Replacement
# FG - Field Goals
# FGA - Field Goal Attempts
# FG% - Field Goal Percentage
# 3P - 3-Point Field Goals
# 3PA - 3-Point Field Goal Attempts
# 3P% - 3-Point Field Goal Percentage
# 2P - 2-Point Field Goals
# 2PA - 2-Point Field Goal Attempts
# 2P% - 2-Point Field Goal Percentage
# eFG% - Effective Field Goal Percentage
# FT - Free Throws
# FTA - Free Throw Attempts
# FT% - Free Throw Percentage
# ORB - Offensive Rebounds
# DRB - Defensive Rebounds
# TRB - Total Rebounds
# AST - Assists
# STL - Steals
# BLK - Blocks
# TOV - Turnovers
# PF - Personal Fouls
# PTS - Points

# https://www.basketball-reference.com/about/glossary.html for additional data

# use the past 12 years --> 2005 - 2017


In [5]:
df_full.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [6]:
df=df_full[df_full['Year']>=2005]

In [7]:
df.shape

(7603, 52)

In [8]:
df_full.shape

(24691, 52)

In [9]:
# df_coach=pd.read_csv('data/NBA_head_coaches.csv.xls')

In [10]:
# df_coach.info()

In [11]:
# splitting the teams into a list of years

In [12]:
# df_coach['Teams']=df_coach['Teams'].str.split(',')

In [13]:
# df.head()

In [14]:
# df_coach[df_coach['Name']=='Rick Adelman']

In [15]:
df['Year']=df['Year'].astype(int)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
#Scarping for coaches

In [17]:
 req = requests.get('https://www.basketball-reference.com/coaches/NBA_stats.html')

In [18]:
content=req.content

In [19]:
soup=BeautifulSoup(content) 

In [20]:
table=soup.find('table');

In [28]:
table.find_all('a')[1].text

'Richie Adubato'

In [22]:
coach_name=[]
for element in table.find_all('a'):
    coach_name.append(element.text)
    

In [24]:
table.find('a').attrs['href']

'/coaches/adelmri01c.html'

In [25]:
table.find_all('a')[0].attrs['href']

'/coaches/adelmri01c.html'

In [32]:
len(table.find_all('a'))

329

In [26]:
#scarping all of the coach tables

In [34]:
coach_list=[]
for element in table.find_all('a'):
    loc='https://www.basketball-reference.com/' + element.attrs['href']
    raw_coach_tables=requests.get(loc)
    raw_coach_table_content=raw_coach_tables.content
    soup2=BeautifulSoup(raw_coach_table_content) 
    coach_table_content=soup2.find('table')
    coach_list.append(pd.read_html(str(coach_table_content)))
   

In [269]:
sorted(coach_name);

In [28]:
#cleaning and merging coach table:

In [37]:

for idx,element in enumerate(coach_list):
#     element[0]=element[0][element[0]['Unnamed: 4_level_0']['G'].astype(str)!='Assistant Coach']
    element[0]['coach_name']=coach_name[idx]
    if idx==0:
        all_coach=element[0]
    else:
        all_coach=all_coach.append(element[0])

In [ ]:
# coach_columns={Team: 'Playoffs', 'Tm', Coach_Age: 'Regular Season','Age' , Season: 'Unnamed: 0_level_0','Season', ---,
#                Playoff_games'Unnamed: 11_level_0', 'G' ,---, playoff_wins: 'Unnamed: 12_level_0', 'W', ---,playoff_loses: 'Unnamed: 13_level_0', 'L'
#               ,---, playoff_W/L%: 'Unnamed: 14_level_0', 'W/L%', --- ,season_games:'Unnamed: 4_level_0', 'G', ---, season_wins: 'Unnamed: 5_level_0', 'W',
#               ---,season_loses: 'Unnamed: 6_level_0', 'L', ---, season_W/L% : 'Unnamed: 7_level_0', 'W/L%', --- ,'Unnamed: 8_level_0', 'W > .500',
#               ---, 'Unnamed: 9_level_0', 'Finish'}

In [44]:
len(all_coach.columns)

226

In [105]:
# for element in range(226):
#     print(all_coach.columns[element])
#     print(all_coach[all_coach.columns[element][0]].head())

In [45]:
all_coach.head()

,Playoffs,Regular Season,Unnamed: 0_level_0,Unnamed: 100_level_0,Unnamed: 101_level_0,Unnamed: 102_level_0,Unnamed: 103_level_0,Unnamed: 104_level_0,Unnamed: 105_level_0,Unnamed: 106_level_0,...,Unnamed: 92_level_0,Unnamed: 93_level_0,Unnamed: 94_level_0,Unnamed: 95_level_0,Unnamed: 96_level_0,Unnamed: 97_level_0,Unnamed: 98_level_0,Unnamed: 99_level_0,Unnamed: 9_level_0,coach_name
,Tm,Age,Season,Unnamed: 100_level_1,Unnamed: 101_level_1,Unnamed: 102_level_1,Unnamed: 103_level_1,Unnamed: 104_level_1,Unnamed: 105_level_1,Unnamed: 106_level_1,...,Unnamed: 92_level_1,Unnamed: 93_level_1,Unnamed: 94_level_1,Unnamed: 95_level_1,Unnamed: 96_level_1,Unnamed: 97_level_1,Unnamed: 98_level_1,Unnamed: 99_level_1,Finish,
0,POR,37.0,1983-84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rick Adelman
1,POR,38.0,1984-85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rick Adelman
2,POR,39.0,1985-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rick Adelman
3,POR,40.0,1986-87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rick Adelman
4,POR,41.0,1987-88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rick Adelman


In [91]:
df_coach=pd.DataFrame()

In [121]:
df_coach['Team']=all_coach['Playoffs']['Tm']

In [93]:
df_coach['coach_age']=all_coach['Regular Season']['Age']

In [94]:
df_coach['coach_name']=all_coach['coach_name']

In [95]:
df_coach['season']=all_coach['Unnamed: 0_level_0']['Season']

In [96]:
df_coach['playoff_games']=all_coach['Unnamed: 11_level_0']['G']

In [97]:
df_coach['playoff_wins']=all_coach['Unnamed: 12_level_0']['W']

In [98]:
df_coach['playoff_loses']=all_coach['Unnamed: 13_level_0']['L']

In [99]:
df_coach['playoff_W/L%']=all_coach['Unnamed: 14_level_0']['W/L%']

In [100]:
df_coach['season_games']=all_coach['Unnamed: 4_level_0']['G']

In [101]:
df_coach['season_wins']=all_coach['Unnamed: 5_level_0']['W']

In [102]:
df_coach['season_loses']=all_coach['Unnamed: 6_level_0']['L']

In [103]:
df_coach['season_W/L%']=all_coach['Unnamed: 7_level_0']['W/L%']

In [104]:
df_coach['season_position']=all_coach['Unnamed: 9_level_0']['Finish']

In [82]:
df_coach.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3982 entries, 0 to 5
Data columns (total 13 columns):
Team               3653 non-null object
coach_age          3180 non-null float64
coach_name         3982 non-null object
season             3982 non-null object
playoff_games      1094 non-null float64
playoff_wins       1094 non-null float64
playoff_loses      1094 non-null float64
playoff_W/L%       1094 non-null float64
season_games       3982 non-null object
season_wins        2149 non-null float64
season_loses       2149 non-null float64
season_W/L%        2149 non-null float64
season_position    1818 non-null float64
dtypes: float64(9), object(4)
memory usage: 435.5+ KB


In [184]:
#remove coaches before season 2005 and remove years = 'Career'

In [154]:
df_coach.dropna(axis=0,inplace=True) #dropped assistant coaches

In [173]:
df_coach[df_coach['coach_name']=='Kenny Atkinson']; #check for phil Jackson

In [223]:
#remove coach before 2004

# Continue from here

In [81]:
# check to see all the data is in the table?

In [169]:
temp=[]
for element in coach_name:
    if element not in df_coach['coach_name'].values:
        temp.append(element)
        

In [176]:
sorted(temp)

['Albert Soar',
 'Alvin Julian',
 'Andy Phillip',
 'Bill Berry',
 'Bill Bertka',
 'Bill Cartwright',
 'Bill Hanzlik',
 'Bill Musselman',
 'Bob Cousy',
 'Bob Hopkins',
 'Bob Kauffman',
 'Bob Kloppenburg',
 'Bob Lanier',
 'Bob Pettit',
 'Bob Staak',
 'Brendan Malone',
 'Brian Shaw',
 'Brian Winters',
 'Bruce Hale',
 'Bucky Buckwalter',
 'Burl Friddle',
 'Butch Beard',
 'Carl Bennett',
 'Carl Braun',
 'Charles Eckman',
 'Charles Wolf',
 'Charley Shipp',
 'Chick Reiser',
 'Chris Jent',
 'Curly Armstrong',
 'Darrell Walker',
 'Dave DeBusschere',
 'Dave McMillan',
 'Dennis Johnson',
 'Derek Fisher',
 'Dick Fitzgerald',
 'Dick Harter',
 'Dick Van Arsdale',
 'Dick Versace',
 'Dick Vitale',
 'Don Casey',
 'Don Delaney',
 'Draff Young',
 'Dutch Dehnert',
 'Earl Lloyd',
 'Earl Watson',
 'Ed Gregory',
 'Ed Jucker',
 'Ed Sadowski',
 'Ed Tapscott',
 'Eddie Donovan',
 'Elgin Baylor',
 'Eric Musselman',
 'Frank Hamblen',
 'Frank McGuire',
 'Fred Carter',
 'Fred Scolari',
 'Gar Heard',
 'Garry St. Jean